# NLP🤗

In [ ]:
from transformers import pipeline
from transformers import AutoModel

In [2]:
# load the model
model_name = "facebook/bart-large-cnn"
model = AutoModel.from_pretrained(model_name)

In [ ]:
# save the model 
save_directory = "./nlp-huggingFace-models/facebookbart-large-cnn"
# Save the model
model.save_pretrained(save_directory)

In [2]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [4]:
import pandas as pd
dataframe = pd.read_excel('bookInfo.xlsx')

In [5]:
dataframe

,bookName,author,publishDate,genres,summary
0,Animal Farm,George Orwell,1945-08-17,"['Roman', 'Satire', ""Children's literature"", '...","Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,1962,"['Science Fiction', 'Novella', 'Speculative fi...","Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,1947,"['Existentialism', 'Fiction', 'Absurdist ficti...",The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,David Hume,NaN,['Philosophy'],The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,Vernor Vinge,NaN,"['Hard science fiction', 'Science Fiction', 'S...",The novel posits that space around the Milky ...
...,...,...,...,...,...
16554,Under Wildwood,Colin Meloy,2012-09-25,['Fantasy'],"Prue McKeel, having rescued her brother from ..."
16555,Transfer of Power,Vince Flynn,2000-06-01,"['Thriller', 'Fiction']",The reader first meets Rapp while he is doing...
16556,Decoded,Jay-Z,2010-11-16,['Autobiography'],The book follows very rough chronological ord...
16557,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,['Non-Fiction'],Colbert addresses topics including Wall Stree...


In [7]:
print(summarizer(dataframe["summary"][11], do_sample=False))

[{'summary_text': 'The Book of Ezra consists of ten chapters. chapters 1-6 cover the period from the Decree of Cyrus to the dedication of the Second Temple. chapters 7-10, dealing with the mission of Ezra, are told largely in the first person. The book contains several documents presented as historical inclusions.'}]
